In [0]:
%run ../utils/common

In [0]:
dbutils.widgets.text("environment", "", "")
dbutils.widgets.text("target_dataset", "VCM_DMT_PRD")
dbutils.widgets.text("target_table", "a_billcnt_mch4")
dbutils.widgets.text("metadata_schema", "udp_wcm_metadata_dev")
dbutils.widgets.text(
    "dependency_table","VCM_DWH_PRD.F_TRANSACTION_LINE, VCM_STG_INF.SAP_VCDEXT01, VCM_DMT_PRD.A_FINANCE_BILL_SLL_DETAIL"
)

In [0]:
dbutils.widgets.text("field_calday", "CALDAY")
dbutils.widgets.text("field_id", "HASH_ID")

In [0]:
dbutils.widgets.text("environment", "", "")
environment = dbutils.widgets.get("environment")

In [0]:
catalog_name = settings[environment]['catalog_name']

In [0]:
environment = dbutils.widgets.get("environment")
target_dataset = dbutils.widgets.get("target_dataset")
target_table = dbutils.widgets.get("target_table")
metadata_schema = dbutils.widgets.get("metadata_schema")
dependency_table = dbutils.widgets.get("dependency_table")
dependency_table = [x.strip().upper() for x in dependency_table.split(",")]
dependency_table = "'" + "','".join(dependency_table) + "'"

print(f"environment: {environment}")
print(f"target_dataset: {target_dataset}")
print(f"target_table: {target_table}")
print(f"catalog_name: {catalog_name}")
print(f"metadata_schema: {metadata_schema}")
print(f"dependency_table: {dependency_table}")

In [0]:
field_calday = dbutils.widgets.get("field_calday")
field_id = dbutils.widgets.get("field_id")

print(f"field_calday: {field_calday}")
print(f"field_id: {field_id}")

In [0]:
%run "../common/common_etl_load"

In [0]:
spark.sql(f"""
CREATE TABLE IF NOT EXISTS {catalog_name}.udp_wcm_gold_vcm_dmt.a_billcnt_mch4 (
transaction_id STRING,
calday DATE,
store_id STRING,
store_name STRING,
mch4_id STRING,
mch4_desc STRING,
billcnt INT,
billcnt_excl_sll INT,
billcnt_promo INT,
billcnt_store INT
)
TBLPROPERTIES (
  'DELTA.AUTOOPTIMIZE.OPTIMIZEWRITE' = 'TRUE',
  'DELTA.AUTOOPTIMIZE.AUTOCOMPACT' = 'TRUE'
)
""")

In [0]:
if CALDAY_IN_STR != '':
    spark.sql(f"""
    CREATE OR REPLACE TEMP VIEW temp_a_billcnt_mch4 AS
    WITH transaction_sll AS (
        SELECT
            DISTINCT
            calday,
            plant AS store_id,
            rpa_tnr AS transaction_no
        FROM {catalog_name}.udp_wcm_silver_sap_bw.sap_vcdext01
        WHERE _bic_zc_fgroup = 'OGID'
            AND _bic_zc_fname = 'B2B'
            AND _bic_zc_fvalue LIKE 'B2B%'
    ),
    store_txn AS (
        SELECT
            a.calday,
            a.store_id,
            b.mch4_id,
            b.mch4_desc,
            a.transaction_type,
            COUNT(DISTINCT transaction_id) AS m_txn_cnt,
            COUNT(DISTINCT IF(c.calday IS NULL AND c1.calday IS NULL, transaction_id, NULL)) AS m_txn_cnt_excl_sll,
            COUNT(DISTINCT CASE WHEN promotion_sale > 0 THEN transaction_id END) AS m_txn_cnt_promo
        FROM
            {catalog_name}.udp_wcm_gold_vcm_dwh.f_transaction_line a
            INNER JOIN {catalog_name}.udp_wcm_gold_vcm_dwh.d_product b ON a.product_id = b.product_id
            LEFT JOIN {catalog_name}.udp_wcm_gold_vcm_dmt.a_finance_bill_sll_detail c ON a.calday = c.calday AND a.store_id = c.store_id AND a.product_id = c.product_id AND LTRIM('0', a.transaction_no) = LTRIM('0', c.transaction_no)
            LEFT JOIN transaction_sll c1 ON a.calday = c1.calday AND a.store_id = c1.store_id AND LTRIM('0', a.transaction_no) = LTRIM('0', c1.transaction_no)
        WHERE a.calday IN ({CALDAY_IN_STR})
        GROUP BY 1,2,3,4,5
    ),
    bill_cnt_by_mch4 AS (
        SELECT
            calday,
            a.store_id,
            a.mch4_id,
            a.mch4_desc,
            SUM(b.bill_count * a.m_txn_cnt) AS billcnt_total,
            SUM(b.bill_count * a.m_txn_cnt_excl_sll) AS billcnt_excl_sll,
            SUM(b.bill_count * a.m_txn_cnt_promo) AS billcnt_promo
        FROM
            store_txn a
            INNER JOIN {catalog_name}.udp_wcm_silver_dim.transaction_type b ON a.transaction_type = b.transaction_type
        GROUP BY 1,2,3,4
    )
    SELECT
        a.calday,
        a.store_id,
        b.store_name,
        a.mch4_id,
        a.mch4_desc,
        a.billcnt_total AS billcnt,
        a.billcnt_excl_sll,
        a.billcnt_promo,
        b.billcnt_total AS billcnt_store
    FROM bill_cnt_by_mch4 a
    LEFT JOIN {catalog_name}.udp_wcm_gold_vcm_dmt.a_billcnt_store b ON a.calday = b.calday AND a.store_id = b.store_id
    """)
else:
    print(f"No valid calday's found In common-etl to create the view: temp_{target_table}")

In [0]:
if CALDAY_IN_STR != "":
    # Get the calday list string from temp table
    calday_query = """
    SELECT
        CONCAT("'", CONCAT_WS("','", COLLECT_SET(CAST(TO_DATE(calday, 'yyyy-MM-dd') AS STRING))), "'") AS calday_in_str
    FROM temp_a_billcnt_mch4
    """

    calday_df = spark.sql(calday_query)
    calday_in_str = calday_df.collect()[0][0]

    # Only run DELETE if the string is non-empty
    if calday_in_str.strip("'"):  # checks if there's any date inside the quotes
        spark.sql(f"""
        DELETE FROM {catalog_name}.udp_wcm_gold_vcm_dmt.a_billcnt_mch4
        WHERE calday IN ({calday_in_str})
        """)
    else:
        print("No valid CALDAYs found to delete.")
else:
    print(
        f"temp_{target_table} could't be created as no valid calday's found In common-etl so {catalog_name}.udp_wcm_gold_vcm_dwh.a_billcnt_store can't be deleted using calday's from temp_a_billcnt_store"
    )

In [0]:
if CALDAY_IN_STR != "":
    # Insert rows from temp table into main table
    INSERT_ROW = ", ".join(
        spark.table(f"{catalog_name}.udp_wcm_gold_vcm_dmt.a_billcnt_mch4").columns
    )
    spark.sql(
        f"""
        INSERT INTO {catalog_name}.udp_wcm_gold_vcm_dmt.a_billcnt_mch4 ({INSERT_ROW})
        SELECT {INSERT_ROW} FROM temp_{target_table}
    """
    ).display()
else:
    print(
        f"temp_{target_table} could't be created as no valid calday's found In common-etl so {catalog_name}.udp_wcm_gold_vcm_dmt.a_billcnt_mch4 can't be inserted using calday's from temp_{target_table}"
    )

In [0]:
if CALDAY_IN_STR == '':
    dbutils.notebook.exit("No valid calday's found In common-etl to insert data into Table: ETL_LOG_DEPENDENCIES and ETL_DELTA_TABLE and ")

In [0]:
%run "../common/common_etl_update"